## Задание 2
### Создам себе тестовую бд для проверки:

In [1]:
# Подключаюсь к учебной БД
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

group_schema = 'gu_game_analytics_1188'

In [2]:
import pandas as pd

In [3]:
# Создаю таблицы
with engine.connect() as conn:
    conn.execute(f'''
        CREATE TABLE {group_schema}.payment (
        time date,
        user_id  INT,
        mobile INT,
        transaction_id INT,
        rev float
        )
    ''')
    


In [4]:
with engine.connect() as conn:
    conn.execute(f'''
        CREATE TABLE {group_schema}.users (
        user_id  INT,
        first_name TEXT,
        age INT,
        reg_time date
        )
    ''')

In [5]:
# Проверяю тип данных
df = pd.read_sql(f'''
    SELECT *
    FROM information_schema.columns
    WHERE table_name='payment'
''', con=engine)
df


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,game-analytics,gu_game_analytics_1188,payment,time,1,None,YES,date,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,game-analytics,gu_game_analytics_1188,payment,user_id,2,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,game-analytics,gu_game_analytics_1188,payment,mobile,3,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,game-analytics,gu_game_analytics_1188,payment,transaction_id,4,None,YES,integer,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,game-analytics,gu_game_analytics_1188,payment,rev,5,None,YES,double precision,None,None,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [6]:
# Придумываю рандомные значения, для дальнейшей проверки
with engine.connect() as conn:
    conn.execute(f'''
        INSERT INTO {group_schema}.payment
        VALUES
        ('2022-06-17', 1, 0, 123, 3454.50),
        ('2022-06-17', 2, 1, 321, 255.70),
        ('2022-07-17', 3, 1, 322, 354.21),
        ('2022-06-18', 1, 1, 323, 4214224.10),
        ('2022-06-19', 1, 1, 324, 4214224.10),
        ('2022-06-20', 2, 1, 325, 4224.10),
        ('2022-06-18', 1, 1, 326, 4.10),
        ('2022-06-18', 5, 1, 327, 42224.10)
    ''')


In [7]:
with engine.connect() as conn:
    conn.execute(f'''
        INSERT INTO {group_schema}.users
        VALUES
        (1, 'паша', 23, '2022-06-17'),
        (2, 'маша', 21, '2022-06-17'),
        (3, 'даша', 22, '2022-06-19'),
        (4, 'глаша', 23, '2022-06-19'),
        (5, 'ваша', 24, '2022-06-19'),
        (6, 'наша', 25, '2022-06-19'),
        (7, 'раша', 26, '2022-06-19'),
        (8, 'аша', 27, '2022-06-19')
    ''')

In [8]:
# Выгружаю созданные таблицы:
payment = pd.read_sql(f'''SELECT * FROM {group_schema}.payment''',con=engine)
payment


,time,user_id,mobile,transaction_id,rev
0,2022-06-17,1,0,123,3454.50
1,2022-06-17,2,1,321,255.70
2,2022-07-17,3,1,322,354.21
3,2022-06-18,1,1,323,4214224.10
4,2022-06-19,1,1,324,4214224.10
5,2022-06-20,2,1,325,4224.10
6,2022-06-18,1,1,326,4.10
7,2022-06-18,5,1,327,42224.10


In [9]:
users = pd.read_sql(f'''SELECT * FROM {group_schema}.users''',con=engine)
users


,user_id,first_name,age,reg_time
0,1,паша,23,2022-06-17
1,2,маша,21,2022-06-17
2,3,даша,22,2022-06-19
3,4,глаша,23,2022-06-19
4,5,ваша,24,2022-06-19
5,6,наша,25,2022-06-19
6,7,раша,26,2022-06-19
7,8,аша,27,2022-06-19


# Тестовые таблицы создал, теперь можно приступать к заданию:

In [10]:
# Привожу к нужному типу и сокращаю даты до дня
import datetime as dt
payment['time'] = pd.to_datetime(payment['time'])
payment['time'] = payment['time'].dt.to_period('D')

# Решение

In [11]:
df = pd.read_sql(f'''
    SELECT DISTINCT time, sum(rev), count(rev), count (distinct p.user_id)
    FROM {group_schema}.payment as p
    LEFT JOIN {group_schema}.users as u
    ON p.user_id = u.user_id
    WHERE 1 = 1
        AND mobile = 1
        AND time < (CURRENT_DATE - 7)
    GROUP BY time
''',con=engine)
df

,time,sum,count,count
0,2022-06-17,255.7,1,1
1,2022-06-18,4256452.3,3,2
2,2022-06-19,4214224.1,1,1
3,2022-06-20,4224.1,1,1


In [12]:
# Переименуем колонки для окончательного ответа:
df.columns = ['дата','Сумма покупок','Кол-во платежей','Кол-во плательщиков']
df

,дата,Сумма покупок,Кол-во платежей,Кол-во плательщиков
0,2022-06-17,255.7,1,1
1,2022-06-18,4256452.3,3,2
2,2022-06-19,4214224.1,1,1
3,2022-06-20,4224.1,1,1


In [13]:
# Удаляю созданные таблицы, чтобы не нагружать учебный сервер:
with engine.connect() as conn:
    conn.execute(f'''
    DROP TABLE IF EXISTS {group_schema}.payment;
    DROP TABLE IF EXISTS {group_schema}.users;
    ''')

